In [448]:
import numpy as np
import queue
from functools import reduce
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [449]:
def make_chains(mat, names, k, key):
    '''Передаём key, чтобы словарь поиска содеражал такие ключи'''
    def dfs(que, mat, v, k_now, k_max, chainsDict, names):
        if k_now == k_max:
            name_in_num = list(que)
            name_in_num.sort()
            name_in_num = list(map(lambda i: names[i], name_in_num))
            kk = reduce(lambda i, j: i + j, name_in_num)
            chainsDict.setdefault(kk, 0)
            chainsDict[kk] += 1
            return
        
        for i in range(0, mat.shape[0]):
            if mat[v, i] == 1 and not i in que:
                que.append(i)
                dfs(que, mat, i, k_now + 1, k_max, chainsDict, names)
                del(que[k_now])
    
    
            
    chainsDict = dict.fromkeys(key, 0)
    
    for i in range(mat.shape[0]):#Запускаем обход в глубину k из каждой вершины
        que = []
        que.append(i)
        
        dfs(que, mat, i, 1, k, chainsDict, names)
        del(que[0])
        
        
    for i in chainsDict.keys(): #Каждую цепочку посчитали два раза
        chainsDict[i] //= 2
    return chainsDict
        

In [525]:
def getAllKChainsNameInFile(filename, k): #k-размер цепочек
    all_chains_names = []
    ctr = 0
    with  open(filename, "r") as f:
        
        
        for i in f:
            ctr += 1
            mol_atoms = []
            mol_name = i.split()[1]
            num_elems = int(i.split()[0])
            

            matrix = np.zeros((num_elems, num_elems))
            for i in range(num_elems):
                
                line = next(f) #Берём i линию в молекуле
                
                mol_atoms.append(line.split()[0]) #По первому индексу хранятся названия, которые пихаем в лист

                it = 3 #С третьего элемента в строке начинаются связи
                while line.split()[it] != "00":
                    matrix[i, int(line.split()[it][:-1]) - 1] = 1 #int(line.split()[it][-1]) Если захотим тип связи
                    matrix[int(line.split()[it][:-1]) - 1, i] = 1
                    it += 1
            res = make_chains(matrix, mol_atoms, k, [])
            all_chains_names += res.keys()
        
    returnList = list(np.unique(all_chains_names))
    returnList.sort()
    return list(np.unique(all_chains_names)), ctr

In [526]:
def getMatrix(filename, k, keys, high):
    res_matr = np.zeros((high, len(keys)))
    dic = dict.fromkeys(keys, 0)
    
    cnt = 0
    with open(filename, "r") as f:
        for i in f:
            
            mol_atoms = []
            num_elems = int(i.split()[0])
            matrix = np.zeros((num_elems, num_elems))
            for i in range(num_elems):
                line = next(f) #Берём i линию в молекуле
                mol_atoms.append(line.split()[0]) #По первому индексу хранятся названия, которые пихаем в лист

                it = 3 #С третьего элемента в строке начинаются связи
                while line.split()[it] != "00":
                    matrix[i, int(line.split()[it][:-1]) - 1] = 1 #int(line.split()[it][-1]) Если захотим тип связи
                    matrix[int(line.split()[it][:-1]) - 1, i] = 1
                    it += 1
            res = make_chains(matrix, mol_atoms, k, dic.keys())
            res_matr[i,:] = list(res.values())
            cnt += 1
    return res_matr

In [527]:
y = []
file_y = "ALKAN.SET"

In [528]:
with open(file_y, "r") as f:
    for i in f:
        splitString = i.split()
        if len(splitString) > 0 and splitString[0] == 'A1':
            y.append(float(splitString[1]))

In [529]:
filename = "ALKAN.STR"
len(y)

74

In [530]:
names, numMolec = getAllKChainsNameInFile(filename, 3)

In [535]:
chainLen = 3

In [536]:
names, numMolec = getAllKChainsNameInFile(filename, chainLen)

In [537]:
Matr = getMatrix(filename, chainLen, names, numMolec)

In [538]:
Matr

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [539]:


X_train, X_test, y_train, y_test = train_test_split(
    Matr, y, test_size=0.4)

In [540]:

clf = LinearRegression()

In [541]:
clf.fit(X_train, y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [542]:
clf.score(X_test, y_test)

-5.822342307769397e+26

In [543]:
list(zip(clf.predict(X_test), y_test))

[(119.58752272727271, 138.0),
 (119.58752272727271, 109.103),
 (119.58752272727271, 63.282),
 (119.58752272727271, 98.427),
 (119.58752272727271, 140.6),
 (119.58752272727271, 117.647),
 (119.58752272727271, 49.741),
 (119.58752272727271, 133.83),
 (5825842100164220.0, 60.271),
 (2548600923749083.5, 27.852),
 (119.58752272727271, 150.798),
 (119.58752272727271, 143.0),
 (119.58752272727271, -88.63),
 (119.58752272727271, 137.68),
 (119.58752272727271, 136.73),
 (119.58752272727271, 106.47),
 (119.58752272727271, 57.988),
 (119.58752272727271, 144.18),
 (119.58752272727271, 117.725),
 (119.58752272727271, 113.467),
 (119.58752272727271, 79.197),
 (119.58752272727271, 118.259),
 (119.58752272727271, 106.84),
 (119.58752272727271, 118.534),
 (119.58752272727271, 133.5),
 (119.58752272727271, 115.65),
 (119.58752272727271, 140.274),
 (6.962522727272713, 68.74),
 (119.58752272727271, 142.0),
 (119.58752272727271, 80.882)]

In [544]:
with open("res.txt", "w") as f:
    print(Matr, file=f)

In [545]:
Matr

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])